In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.http import FormRequest
import pandas as pd


class PostsSpider(scrapy.Spider):
    name = "posts"
    

    start_urls = [
        'https://depositonce.tu-berlin.de/'
    ]

    def __init__(self, query):
        self.query = query
        self.arr=[]
        

    def parse(self, response):
        return FormRequest.from_response(response,
                                         formdata={'query': self.query},
                                         callback=self.search_result
                                         )
    
    
    
    def choose(self, args):
        x = input("choose: ")
        print(args[1])
        
        
    

    def search_result(self, response):
        
        for post in response.css('.table tr:not(:first-child)'):
            
            self.arr.append(
                {
                    'Issue date' : post.css('td:nth-child(2)::text').get(),
                    'Title' : post.css('td:nth-child(3) a::text').get(),
                    'Author' : post.css('td:nth-child(4) a::text').get()
                }
            
            )
            
        next_page = response.css('.pagination li:last-child a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback = self.search_result)
        else:
            
            df = pd.DataFrame(self.arr)
            print(df)
            self.choose(self.arr)
             
            
   
            

        

def main(params):
    process = CrawlerProcess()
    process.crawl(PostsSpider, query = params)
    
    process.start()
    

g = input("input the data: ")
main(g)       